In [2]:
%config Completer.use_jedi = False
%matplotlib inline
import random
import torch
from torch.utils import data

In [5]:
## 生成数据集与手动版
# 使用线性模型参数 w=[2,−3.4]⊤ 、 b=4.2 和噪声项 ϵ 生成数据集及其标签：y=Xw+b+ϵ.
# 可以将  ϵ  视为捕获特征和标签时的潜在观测误差。在这里我们认为标准假设成立，即 ϵ 服从均值为0的正态分布。
# 为了简化问题，我们将标准差设为0.01。下面的代码生成合成数据集。

def synthetic_data(w, b, num_examples):  #@save
    """生成 y = Xw + b + 噪声。"""
    X = torch.normal(0, 1, (num_examples, len(w))) 
    y = torch.matmul(X, w) + b # 1000 X 1
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print(features[1],labels[1])

tensor([-0.8812,  0.4406]) tensor([0.9518])


In [6]:
# 读取数据集
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """构造一个PyTorch数据迭代器。"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [7]:
# 查看迭代取数是否正常
next(iter(data_iter))

[tensor([[-1.4016, -1.6029],
         [-1.0687, -1.6633],
         [ 0.6922, -0.0588],
         [-0.9198, -1.3350],
         [ 2.0441,  0.7863],
         [-0.2757,  0.2868],
         [ 0.3032,  1.5384],
         [-0.9962, -2.2553],
         [-0.0897,  0.0033],
         [ 2.2038, -1.5245]]),
 tensor([[ 6.8442],
         [ 7.7281],
         [ 5.7971],
         [ 6.9003],
         [ 5.6190],
         [ 2.6939],
         [-0.4086],
         [ 9.8762],
         [ 4.0031],
         [13.7947]])]

In [8]:
# 定义模型
# `nn` 是神经网络的缩写
from torch import nn

net = nn.Sequential(nn.Linear(2, 1))

In [13]:
# 初始化模型参数
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)
net[0].bias

Parameter containing:
tensor([0.], requires_grad=True)

In [14]:
# 定义损失函数
# 计算均方误差使用的是MSELoss类，也称为平方  L2  范数。默认情况下，它返回所有样本损失的平均值。
loss = nn.MSELoss()

In [15]:
# 定义优化算法
# 我们要指定优化的参数（可通过 net.parameters() 从我们的模型中获得）以及优化算法所需的超参数字典。小批量随机梯度下降只需要设置 lr值。
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [16]:
# 开始训练
# 通过调用 net(X) 生成预测并计算损失 l（正向传播）。
# 通过进行反向传播来计算梯度。
# 通过调用优化器来更新模型参数。
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad() # 梯度清零
        l.backward() # 反向传播
        trainer.step() # 更新阐述
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000189
epoch 2, loss 0.000100
epoch 3, loss 0.000100


In [17]:
# 和真实的参数比较误差
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： tensor([ 0.0006, -0.0002])
b的估计误差： tensor([0.0007])
